In [64]:
CLIENT_ID = 'MnLMKnyq3_YI43rKVp3_Gw'
SECRET_KEY = 'Bpvh_flU8yAJE4PdUo2Wj-S6uQx6xw'

In [65]:
import requests
import praw
import pandas as pd
from datetime import datetime
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [66]:
reddit = praw.Reddit(client_id = 'MnLMKnyq3_YI43rKVp3_Gw', 
                     client_secret = 'Bpvh_flU8yAJE4PdUo2Wj-S6uQx6xw',
                     username='MyPCIsAPotato123',
                     password='7898527412369',
                     user_agent='tutorialv1')

In [67]:
subreddit = reddit.subreddit('wallstreetbets')

In [68]:
df = pd.DataFrame()

for submission in subreddit.search("TSLA", limit=1000):
    df = df.append({'time': submission.created_utc,
                    'title': submission.title,
                    'body': submission.selftext}, ignore_index=True)
    

                    

In [69]:
df['time'] = df['time'].astype(int)
df['time'] = df['time'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d' + ' ' + '%H:%M:%S'))

In [70]:
df = df[['time', 'title', 'body']]
df

,time,title,body
0,2021-10-29 17:49:43,💸 $1 Million Dollar Tesla Short ($TSLA) 📉,How I plan to **Short Tesla ($TSLA)** with a m...
1,2021-11-13 17:05:09,"Thanks to $TSLA, I’m buying a new wife for Chr...",
2,2021-11-06 20:19:11,My lifesavings are in TSLA please don't Elon🙏🏼,
3,2021-11-02 17:04:02,Holding TSLA for almost 10 years. When do I ge...,
4,2021-11-01 21:12:45,💸 $1 Million Dollar Tesla Short ($TSLA) 📉 (PAR...,&#x200B;\n\nhttps://preview.redd.it/x5yu81lnu1...
...,...,...,...
243,2021-06-24 00:57:47,$TSLA Gang: fully erect,
244,2021-08-02 00:49:58,"Romancing the Wood - a TA/DD of your favorite,...",Did you come to Wallstreetbets today looking f...
245,2021-03-22 22:52:24,Trying to time a TSLA trade,
246,2021-10-25 18:36:33,$TSLA Daddy Elon Delivering Tendies,


In [71]:
df1 = df.sort_values(by='time', ascending=False).reset_index(drop=True)

In [72]:
df1

,time,title,body
0,2021-11-23 23:58:10,In Poppa Elon we trust! What are your projecti...,
1,2021-11-23 19:45:21,ENOUGH RETARD TSLA FUD,All I see is stupid FUDs making a mockery of t...
2,2021-11-23 18:40:54,I love that! 🧘🐂🚀🌜 $TSLA,
3,2021-11-22 17:29:00,150k profit in 30 mins with $TSLA PUTS. TOO EA...,
4,2021-11-22 14:46:17,The $TSLA 12/9 split conspiracy theorists got ...,
...,...,...,...
243,2021-06-20 19:44:59,$TSLA Chart - Anyone Else Notice This?,
244,2021-05-17 18:32:27,Michael Burry Reals 530 Million Bet Against TSLA,
245,2021-05-14 11:50:21,Let's revive the buried WSB culture! GME to TH...,"&#x200B;\n\n[The GME thesis, which I aim to ex..."
246,2021-03-22 22:52:24,Trying to time a TSLA trade,


In [73]:
nltk.download("vader_lexicon")
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\61421\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [74]:
 # Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

In [75]:
title = {
    "compound": [],
    "pos": [],
    "neu": [],
    "neg": [],
    "classification": [],
}


for index, row in df1.iterrows():
    try:
        title_sentiment = analyzer.polarity_scores(row['title'])
        title['compound'].append(title_sentiment['compound'])
        title['pos'].append(title_sentiment['pos'])
        title['neu'].append(title_sentiment['neu'])
        title['neg'].append(title_sentiment['neg'])
        title['classification'].append(get_sentiment(title_sentiment['compound']))
    except AtributeError:
        pass

In [76]:
title_sentiment_df = pd.DataFrame(title)
combined_df = df1.join(title_sentiment_df)

combined_df['time'] = pd.to_datetime(combined_df['time'])

In [77]:
chart_df = combined_df.set_index('time')
chart_df

,title,body,compound,pos,neu,neg,classification
time,,,,,,,
2021-11-23 23:58:10,In Poppa Elon we trust! What are your projecti...,,0.6062,0.124,0.876,0.000,1
2021-11-23 19:45:21,ENOUGH RETARD TSLA FUD,All I see is stupid FUDs making a mockery of t...,-0.6705,0.000,0.267,0.733,-1
2021-11-23 18:40:54,I love that! 🧘🐂🚀🌜 $TSLA,,0.6696,0.600,0.400,0.000,1
2021-11-22 17:29:00,150k profit in 30 mins with $TSLA PUTS. TOO EA...,,0.8814,0.463,0.537,0.000,1
2021-11-22 14:46:17,The $TSLA 12/9 split conspiracy theorists got ...,,-0.5267,0.000,0.815,0.185,-1
...,...,...,...,...,...,...,...
2021-06-20 19:44:59,$TSLA Chart - Anyone Else Notice This?,,0.0000,0.000,1.000,0.000,0
2021-05-17 18:32:27,Michael Burry Reals 530 Million Bet Against TSLA,,0.0000,0.000,1.000,0.000,0
2021-05-14 11:50:21,Let's revive the buried WSB culture! GME to TH...,"&#x200B;\n\n[The GME thesis, which I aim to ex...",0.9230,0.201,0.799,0.000,1


In [78]:
daily_compound = chart_df.compound.resample('D').mean()

In [80]:
a = pd.DataFrame(daily_compound)

In [81]:
a

,compound
time,
2021-02-03,-0.542300
2021-02-04,NaN
2021-02-05,NaN
2021-02-06,NaN
2021-02-07,NaN
...,...
2021-11-19,NaN
2021-11-20,0.000000
2021-11-21,0.000000
